# CSC413 Final Project - Sensitivity Analysis (HMSA)

## Before Running the Code
1. Ensure that the runtime is set to GPU!
2. Ensure that the Colab VM is in the same region as the Google Cloud Storage Buckets (which are `US-multi-region`).
  - This minimizes costs, as Google charges for network egress.
  - The Colab region can be checked by running the cell below.
  - Perform a factory reset of the runtime until the runtime is in the US.

In [1]:
!curl ipinfo.io

{
  "ip": "35.245.1.234",
  "hostname": "234.1.245.35.bc.googleusercontent.com",
  "city": "Washington",
  "region": "Washington, D.C.",
  "country": "US",
  "loc": "38.8951,-77.0364",
  "org": "AS15169 Google LLC",
  "postal": "20045",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

## 1. Setup


### 1.1. Clone Git Repo

In [2]:
!git clone https://github.com/Brian0615/CSC413FinalProject.git
%cd CSC413FinalProject/
!git checkout brian

Cloning into 'CSC413FinalProject'...
remote: Enumerating objects: 405, done.
remote: Counting objects: 100% (405/405), done.
remote: Compressing objects: 100% (326/326), done.
remote: Total 405 (delta 90), reused 344 (delta 62), pack-reused 0
Receiving objects: 100% (405/405), 21.88 MiB | 41.26 MiB/s, done.
Resolving deltas: 100% (90/90), done.
/content/CSC413FinalProject
Branch 'brian' set up to track remote branch 'brian' from 'origin'.
Switched to a new branch 'brian'


###1.2. Download Mapillary Dataset

In [3]:
%cd /content
!mkdir mapillary
%cd mapillary
!mkdir training
!mkdir validation
!mkdir testing

/content
/content/mapillary


In [4]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapi

In [5]:
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/training/images /content/mapillary/training/
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/training/v1.2/instances /content/mapillary/training/
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/training/v1.2/labels /content/mapillary/training/
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/training/v1.2/panoptic /content/mapillary/training/

!gsutil -m cp -r gs://csc413-final-project-mapillary-data/validation/images /content/mapillary/validation/
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/validation/v1.2/instances /content/mapillary/validation/
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/validation/v1.2/labels /content/mapillary/validation/
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/validation/v1.2/panoptic /content/mapillary/validation/

!gsutil -m cp -r gs://csc413-final-project-mapillary-data/testing/images /content/mapillary/testing/

Streaming output truncated to the last 5000 lines.
Copying gs://csc413-final-project-mapillary-data/testing/images/-23xtBM9iYlwqrpC30GdVw.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-2kFqXxlI_8fZWp8N-GQpQ.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-37SFooDIzzppByLKsil-Q.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-7t5ert596XSkeVIE_k1bg.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-40skKVvNcSxu8u2TCkv8A.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-7yh4M2FVZC6jvYzLbFWwA.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-8f-PphflpZSDU6nqcv4hQ.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-AF71jwA3mYt9YEP6ByndA.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-AWGEs2Yc05cEodYX1a0qg.jpg...
Copying gs://csc413-final-project-mapillary-data/testing/images/-C7APwP7xPcZc7XdGt3SEQ.jpg...
Copying g

In [6]:
!gsutil -m cp -r gs://csc413-final-project-mapillary-data/config_v1.2.json /content/mapillary/config.json

Copying gs://csc413-final-project-mapillary-data/config_v1.2.json...
/ [1/1 files][ 13.2 KiB/ 13.2 KiB] 100% Done                                    
Operation completed over 1 objects/13.2 KiB.                                     


### 1.3. Download Weights


In [7]:
!mkdir /content/seg_weights/
!gsutil -m cp gs://csc413-final-project-hmsa-weights/original_weights/mapillary_ocrnet.HRNet_Mscale_fast-rattlesnake.pth /content/seg_weights/mapillary_ocrnet.HRNet_Mscale_fast-rattlesnake.pth
!gsutil -m cp gs://csc413-final-project-hmsa-weights/original_weights/hrnetv2_w48_imagenet_pretrained.pth /content/seg_weights/hrnetv2_w48_imagenet_pretrained.pth

Copying gs://csc413-final-project-hmsa-weights/original_weights/mapillary_ocrnet.HRNet_Mscale_fast-rattlesnake.pth...
/ [1/1 files][551.8 MiB/551.8 MiB] 100% Done                                    
Operation completed over 1 objects/551.8 MiB.                                    
Copying gs://csc413-final-project-hmsa-weights/original_weights/hrnetv2_w48_imagenet_pretrained.pth...
\ [1/1 files][296.2 MiB/296.2 MiB] 100% Done                                    
Operation completed over 1 objects/296.2 MiB.                                    


### 1.4. Install Dependencies

In [8]:
# install basic dependencies
!pip install runx==0.0.6 numpy sklearn h5py jupyter scikit-image pillow piexif cffi tqdm dominate opencv-python nose ninja
!apt-get update
!apt-get install libgtk2.0-dev -y
!rm -rf /var/lib/apt/lists/*

     |████████████████████████████████| 163kB 28.6MB/s 
     |████████████████████████████████| 112kB 14.7MB/s 
     |████████████████████████████████| 645kB 27.1MB/s 
     |████████████████████████████████| 122kB 56.1MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/r

In [9]:
# change cuda version to 10.1
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda  # replace symlink to cuda-11.0 with cuda-10.1

/usr/local


In [10]:
# install apex
%cd /home/
!git clone https://github.com/NVIDIA/apex.git apex
%cd apex
!python setup.py install --cuda_ext --cpp_ext

/home
Cloning into 'apex'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 7939 (delta 5), reused 12 (delta 0), pack-reused 7913
Receiving objects: 100% (7939/7939), 14.02 MiB | 29.60 MiB/s, done.
Resolving deltas: 100% (5406/5406), done.
/home/apex


torch.__version__  = 1.8.1+cu101


setup.py:67: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-inf

### 1.5. Config Setup
 - Inside `CSC413FinalProject/HMSA/config.py`, set the following items:
  ```
  __C.ASSETS_PATH = '/content'
  __C.DATASET.MAPILLARY_DIR = os.path.join(__C.ASSETS_PATH, 'mapillary')
  ```

In [11]:
# check GPU specs
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-41e9baba-bd4b-f418-6d7c-cfa75953b474)


* Since Colab only has one GPU, we need to adjust the evaluation script
* Inside `CSC413FinalProject/HMSA/scripts/eval_mapillary.yml`, set the following:
  ```
  CMD: "python -m torch.distributed.launch --nproc_per_node=1 train.py"
  ```
  (i.e. change `--nproc_per_node=8` to `--nproc_per_node=1`)

## 2. Inference on Mapillary

In [12]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=8c660ee7c0e020be087c2b8dbedfb3065ac22fcce33cd5b49c44e1e687a1179a
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  |     Proc size: 125.1 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total     15109MB


In [13]:
%cd /content/CSC413FinalProject/HMSA/

/content/CSC413FinalProject/HMSA


In [16]:
# dry run (to see full command)
!python -m runx.runx scripts/eval_mapillary.yml -i -n

python -m torch.distributed.launch --nproc_per_node=1 train.py --dataset mapillary --syncbn --apex --fp16 --bs_val 1 --eval val --pre_size 2048 --amp_opt_level O3 --n_scales 0.25,0.5,1.0,2.0 --do_flip --snapshot ASSETS_PATH/seg_weights/mapillary_ocrnet.HRNet_Mscale_fast-rattlesnake.pth --arch ocrnet.HRNet_Mscale --result_dir logs/eval_mapillary/funky-meerkat_2021.04.10_22.11 


In [19]:
# real run
!python -m runx.runx scripts/eval_mapillary.yml -i

None
Global Rank: 0 Local Rank: 0
Torch version: 1.8, 1.8.1+cu101
n scales [0.25, 0.5, 1.0, 2.0]
dataset = mapillary
ignore_label = 65
num_classes = 65
all imgs 2000
Using Cross Entropy Loss
Using Cross Entropy Loss
Loading weights from: checkpoint=/content/seg_weights/mapillary_ocrnet.HRNet_Mscale_fast-rattlesnake.pth
=> init weights from normal distribution
=> loading pretrained model /content/seg_weights/hrnetv2_w48_imagenet_pretrained.pth
Trunk: hrnetv2
Model params = 72.2M
Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        :

In [21]:
# upload results to google cloud bucket
!gsutil -m cp -r /content/CSC413FinalProject/HMSA/logs/eval_mapillary/* gs://csc413-final-project-mapillary-data/inference_results/

Copying file:///content/CSC413FinalProject/HMSA/logs/eval_mapillary/fluffy-chameleon_2021.04.10_20.50/best_checkpoint_ep0.pth [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///content/CSC413FinalProject/HMSA/logs/eval_mapillary/fluffy-chameleon_2021.04.10_20.50/events.out.tfevents.1618087861.9200161572e2.8079.0 [C